In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

from siuba import *

In [2]:
import create_aggregate_stop_frequencies as casf

In [3]:
from update_vars import analysis_date

In [4]:
analysis_date

'2025-11-05'

In [5]:
st, trips = casf.get_st_trips(analysis_date)  # includes lookback
st_prepped = casf.add_route_dir(trips=trips, stop_times=st, analysis_date=analysis_date).pipe(casf.prep_stop_times)

max_arrivals_by_stop_single = st_prepped.pipe(
    casf.stop_times_aggregation_max_by_stop, analysis_date, single_route_dir=True
)

max_arrivals_by_stop_multi = st_prepped.pipe(
    casf.stop_times_aggregation_max_by_stop, analysis_date, single_route_dir=False
)

{'2025-08-20': ['eTrans Schedule', 'Roseville Transit GMV Schedule'], '2025-09-24': ['San Juan Capistrano Trolley Schedule', 'Culver City Schedule'], '2025-10-15': ['Yolobus Schedule', 'Go West Schedule', 'Bay Area 511 Angel Island-Tiburon Ferry Schedule', 'El Monte Schedule', 'Nevada County Schedule']}


/home/jovyan/data-analyses/high_quality_transit_areas/create_aggregate_stop_frequencies.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stop_times["peak"] = stop_times["arrival_hour"].map(peaks_dict)


In [6]:
multi_only_explode = casf.get_explode_multiroute_only(
    max_arrivals_by_stop_single, max_arrivals_by_stop_multi, (casf.HQ_TRANSIT_THRESHOLD, casf.MS_TRANSIT_THRESHOLD)
)

## feed_level_filter (done)

## collinear_filter_feed (done)

In [7]:
am_peak_hrs = list(range(casf.AM_PEAK[0].hour, casf.AM_PEAK[1].hour))
pm_peak_hrs = list(range(casf.PM_PEAK[0].hour, casf.PM_PEAK[1].hour))
both_peaks_hrs = am_peak_hrs + pm_peak_hrs
peaks_dict = {key: "am_peak" for key in am_peak_hrs} | {key: "pm_peak" for key in pm_peak_hrs}

In [8]:
share_counts = {}
multi_only_explode.groupby(["schedule_gtfs_dataset_key", "stop_id"]).apply(
    casf.accumulate_share_count, share_counts=share_counts
)
qualify_dict = {key: share_counts[key] for key in share_counts.keys() if share_counts[key] >= casf.SHARED_STOP_THRESHOLD}

In [9]:
feeds_to_filter = np.unique([key.split("__")[0] for key in qualify_dict.keys()])

In [10]:
frequency_thresholds = (casf.HQ_TRANSIT_THRESHOLD, casf.MS_TRANSIT_THRESHOLD)

In [77]:
feeds_to_filter[2]

'11865a0a240039d4479ded2595c832a5'

In [84]:
st_could_qual, qualify_pairs = casf.feed_level_filter(
    feeds_to_filter[3], multi_only_explode, qualify_dict, st_prepped, frequency_thresholds
)

## filter qualifying stops (done)

## filter all prepare export

In [114]:
def filter_all_prepare_export(
    feeds_to_filter: list,
    multi_only_explode: pd.DataFrame,
    qualify_dict: dict,
    st_prepped: pd.DataFrame,
    max_arrivals_by_stop_single: pd.DataFrame,
    frequency_thresholds: tuple,
):
    """
    Apply collinearity
    """
    #  %%time 90 seconds (on default user) is not too bad!
    all_collinear = pd.DataFrame()
    for gtfs_dataset_key in feeds_to_filter:
        df = casf.collinear_filter_feed(gtfs_dataset_key, multi_only_explode, qualify_dict, st_prepped, frequency_thresholds)
        all_collinear = pd.concat([df, all_collinear])

    single_qual_min = max_arrivals_by_stop_single[(max_arrivals_by_stop_single.am_max_trips_hr >= min((HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD))) &
        (max_arrivals_by_stop_single.pm_max_trips_hr >= min((HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD)))]
    
    single_only_export = single_qual_min.merge(
        all_collinear[["schedule_gtfs_dataset_key", "stop_id"]],
        on=["schedule_gtfs_dataset_key", "stop_id"],
        how="left",
        indicator=True,
    )
    single_only_export = single_only_export[single_only_export["_merge"] == "left_only"].drop(columns=["_merge"]).reset_index(drop=True)
    combined_export = pd.concat([single_only_export, all_collinear])
    combined_export = combined_export.explode("route_dir")
    combined_export["route_id"] = combined_export["route_dir"].str[:-2]

    return combined_export

In [115]:
feeds_to_filter = np.unique([key.split("__")[0] for key in qualify_dict.keys()])

combined_export = filter_all_prepare_export(
    feeds_to_filter,
    multi_only_explode,
    qualify_dict,
    st_prepped,
    max_arrivals_by_stop_single,
    (casf.HQ_TRANSIT_THRESHOLD, casf.MS_TRANSIT_THRESHOLD),
)

,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short
2,076e30b080fdc5501151bd3fb0a37b9e,2206,9,"[901_1, 902_1]",15,3.0,3.75,24,2,True
59,076e30b080fdc5501151bd3fb0a37b9e,6725,9,"[901_1, 902_1]",15,3.0,3.75,24,2,True
60,076e30b080fdc5501151bd3fb0a37b9e,6728,9,"[901_1, 902_1]",15,3.0,3.75,24,2,True
61,076e30b080fdc5501151bd3fb0a37b9e,6734,9,"[901_1, 902_1]",15,3.0,3.75,24,2,True
62,076e30b080fdc5501151bd3fb0a37b9e,6770,9,"[901_1, 902_1]",16,3.0,4.00,25,2,True
69,076e30b080fdc5501151bd3fb0a37b9e,6915,9,"[901_1, 902_1]",15,3.0,3.75,24,2,True


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


...

,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short
81,d4a07b520153bd5c27860835bc42610e,20033,11,"[230-13188_0, 236-13188_0]",15,3.67,3.75,26,2,True
82,d4a07b520153bd5c27860835bc42610e,20034,11,"[230-13188_0, 236-13188_0]",14,3.67,3.50,25,2,True
83,d4a07b520153bd5c27860835bc42610e,20035,11,"[230-13188_0, 236-13188_0]",15,3.67,3.75,26,2,True
130,d4a07b520153bd5c27860835bc42610e,2916,11,"[230-13188_0, 236-13188_0]",15,3.67,3.75,26,2,True
171,d4a07b520153bd5c27860835bc42610e,5814,11,"[230-13188_0, 236-13188_0]",15,3.67,3.75,26,2,True
172,d4a07b520153bd5c27860835bc42610e,5815,11,"[230-13188_0, 236-13188_0]",14,3.67,3.50,25,2,True


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


In [116]:
combined_export1 = casf.filter_all_prepare_export(
    feeds_to_filter,
    multi_only_explode,
    qualify_dict,
    st_prepped,
    max_arrivals_by_stop_single,
    (casf.HQ_TRANSIT_THRESHOLD, casf.MS_TRANSIT_THRESHOLD),
)

,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short
2,076e30b080fdc5501151bd3fb0a37b9e,2206,9,"[901_1, 902_1]",15,3.0,3.75,24,2,True
59,076e30b080fdc5501151bd3fb0a37b9e,6725,9,"[901_1, 902_1]",15,3.0,3.75,24,2,True
60,076e30b080fdc5501151bd3fb0a37b9e,6728,9,"[901_1, 902_1]",15,3.0,3.75,24,2,True
61,076e30b080fdc5501151bd3fb0a37b9e,6734,9,"[901_1, 902_1]",15,3.0,3.75,24,2,True
62,076e30b080fdc5501151bd3fb0a37b9e,6770,9,"[901_1, 902_1]",16,3.0,4.00,25,2,True
69,076e30b080fdc5501151bd3fb0a37b9e,6915,9,"[901_1, 902_1]",15,3.0,3.75,24,2,True


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


...

,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short
81,d4a07b520153bd5c27860835bc42610e,20033,11,"[230-13188_0, 236-13188_0]",15,3.67,3.75,26,2,True
82,d4a07b520153bd5c27860835bc42610e,20034,11,"[230-13188_0, 236-13188_0]",14,3.67,3.50,25,2,True
83,d4a07b520153bd5c27860835bc42610e,20035,11,"[230-13188_0, 236-13188_0]",15,3.67,3.75,26,2,True
130,d4a07b520153bd5c27860835bc42610e,2916,11,"[230-13188_0, 236-13188_0]",15,3.67,3.75,26,2,True
171,d4a07b520153bd5c27860835bc42610e,5814,11,"[230-13188_0, 236-13188_0]",15,3.67,3.75,26,2,True
172,d4a07b520153bd5c27860835bc42610e,5815,11,"[230-13188_0, 236-13188_0]",14,3.67,3.50,25,2,True


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


,schedule_gtfs_dataset_key,stop_id,am_max_trips,route_dir,pm_max_trips,am_max_trips_hr,pm_max_trips_hr,n_trips,route_dir_count,all_short


In [117]:
combined_export.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24849 entries, 0 to 23
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   schedule_gtfs_dataset_key  24849 non-null  object 
 1   stop_id                    24849 non-null  object 
 2   am_max_trips               24849 non-null  int64  
 3   route_dir                  24849 non-null  object 
 4   pm_max_trips               24849 non-null  float64
 5   am_max_trips_hr            24849 non-null  float64
 6   pm_max_trips_hr            24849 non-null  float64
 7   n_trips                    24849 non-null  float64
 8   route_dir_count            24849 non-null  int64  
 9   route_id                   24849 non-null  object 
dtypes: float64(4), int64(2), object(4)
memory usage: 2.1+ MB


In [118]:
combined_export1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24849 entries, 303 to 23
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   schedule_gtfs_dataset_key  24849 non-null  object 
 1   stop_id                    24849 non-null  object 
 2   am_max_trips               24849 non-null  int64  
 3   route_dir                  24849 non-null  object 
 4   pm_max_trips               24849 non-null  float64
 5   am_max_trips_hr            24849 non-null  float64
 6   pm_max_trips_hr            24849 non-null  float64
 7   n_trips                    24849 non-null  float64
 8   route_dir_count            24849 non-null  int64  
 9   route_id                   24849 non-null  object 
dtypes: float64(4), int64(2), object(4)
memory usage: 2.1+ MB


In [121]:
combined_export = combined_export.reset_index(drop=True)

In [122]:
combined_export1 = combined_export1.reset_index(drop=True)

In [124]:
combined_export.equals(combined_export1)

True